In [150]:
import os
import time
import pandas as pd  
import numpy as np
import re
import sqlite3

The tables in basketball_data are
 - game_data
 - team_data
 - name_xwalk

In [151]:
conn = sqlite3.connect("basketball_data.db")
game_data = pd.read_sql_query("select * from game_data;", conn)
team_data = pd.read_sql_query("select * from team_data;", conn)
name_xwalk = pd.read_sql_query("select * from name_xwalk;", conn)

conn.close()   

### Merge on the CleanName so all the tables speak to each other

In [152]:
name_xwalk_for_game_data = name_xwalk[['url_name','CleanName','Opp']].drop_duplicates(['CleanName'], keep='first').copy()
game_data_clean = pd.merge(game_data,name_xwalk_for_game_data[['url_name','CleanName']],left_on='Tm',right_on='url_name',how='left')
game_data_clean.drop(['Tm','url_name','index'],axis=1,inplace=True)
game_data_clean=game_data_clean.rename(columns = {'CleanName':'Tm'})
game_data_clean = pd.merge(game_data_clean,name_xwalk_for_game_data[['Opp','CleanName']],on='Opp',how='left')
game_data_clean.drop(['Opp'],axis=1,inplace=True)
game_data_clean=game_data_clean.rename(columns = {'CleanName':'Opp'})

In [153]:
name_xwalk_for_team_data = name_xwalk[['Team','CleanName',]].drop_duplicates(['Team','CleanName'], keep='first').copy()
team_data_clean = pd.merge(team_data,name_xwalk[['Team','CleanName']],on='Team',how='left')
team_data_clean.drop(['Team','index'],axis=1,inplace=True)
team_data_clean=team_data_clean.rename(columns = {'CleanName':'Team'})

### Crosswalk my data to Kaggle data  

Kaggle provides a lot of data in their March Madness Learning Mania competition. I'd like to include that data with the data I scraped, so I have to do more cleaning of the names before I can merge them. I did most of this manually in excel (see the CrosswalkNames excel file).

In [154]:
kaggle_names = pd.read_csv("Data/Teams.csv")
kaggle_names.Team_Name = kaggle_names.Team_Name.str.upper()
kaggle_names.loc[:,'Team_Name_Clean'] = kaggle_names.Team_Name
kaggle_names.ix[kaggle_names.Team_Name_Clean == 'W SALEM ST','Team_Name_Clean'] = 'WINSTON SALEM'
kaggle_names.Team_Name_Clean.replace(to_replace=" ST$",value=" STATE",regex=True,inplace=True)

In [155]:
kaggle_xwalk = pd.read_excel("Data/CrosswalkNames.xlsx")[['Team_Id','Team_Name','Team_Name_Clean','Change to Make']].dropna(axis=0,how="all")
kaggle_xwalk = pd.merge(kaggle_names,kaggle_xwalk,on=['Team_Id','Team_Name',"Team_Name_Clean"],how="outer")
kaggle_xwalk.loc[:,'Team_Name_Clean'] = np.where(kaggle_xwalk['Change to Make'].isnull(),kaggle_xwalk['Team_Name_Clean'],kaggle_xwalk['Change to Make'])
kaggle_xwalk.drop('Change to Make',axis=1,inplace=True)

In [156]:
name_xwalk = pd.merge(kaggle_xwalk,name_xwalk,left_on="Team_Name_Clean",right_on="CleanName",how="outer").drop('index',axis=1)
name_xwalk.ix[name_xwalk.CleanName.isnull(),'CleanName']=name_xwalk['Team_Name_Clean']
name_xwalk.drop('Team_Name_Clean',axis=1,inplace=True)

##### The following two cells were used to output the table I used to manually build the crosswalk

misses = new_names[((new_names.Team_Id.notnull()) & (new_names.CleanName.isnull())) | ((new_names.Team_Id.isnull()) & (new_names.url_name.notnull()))].copy()
misses.loc[:,"Sorter"] = np.where(misses.Team_Name_Clean.isnull(),misses.CleanName,misses.Team_Name_Clean)
misses = misses.sort_values('Sorter')

writer = pd.ExcelWriter('CrosswalkNames.xlsx',engine='xlsxwriter')
misses.to_excel(writer,startcol=0,startrow=0,sheet_name='Sheet1',index=False) #index=True will write the index which I almost never want
writer.save()


# Import the Kaggle Data

#### seasons - day number, regions

In [157]:
seasons = pd.read_csv("Data/Seasons.csv")
seasons['Dayzero'] = pd.to_datetime(seasons.Dayzero,format='%m/%d/%Y')
seasons.head()

,Season,Dayzero,Regionw,Regionx,Regiony,Regionz
0,1985,1984-10-29,East,West,Midwest,Southeast
1,1986,1985-10-28,East,Midwest,Southeast,West
2,1987,1986-10-27,East,Southeast,Midwest,West
3,1988,1987-11-02,East,Midwest,Southeast,West
4,1989,1988-10-31,East,West,Midwest,Southeast


#### TourneySeeds

In [158]:
seeds = pd.read_csv("Data/TourneySeeds.csv")
seeds.loc[:,'SeedNum'] = seeds.Seed.str[1:3].astype(int)
seeds.loc[:,'Region'] = seeds.Seed.str[0]
slots = pd.read_csv("Data/TourneySlots.csv")
slots.loc[:,'Round'] = slots.Slot.str[1:2].astype(int)
#slots.loc[:,'Region'] = np.where(slots.Round < 5,slots.Slot.str[2:3],slots.Slot.str[2:4])
#slots.loc[:,'Game'] = np.where(slots.Round < 5,slots.Slot.str[-1],slots.Slot.str[2:4])

#### Game results

In [159]:
season_results_compact = pd.read_csv("Data/RegularSeasonCompactResults.csv")
season_results_detailed = pd.read_csv("Data/RegularSeasonDetailedResults.csv")
tourney_results_compact = pd.read_csv("Data/TourneyCompactResults.csv")
tourney_results_detailed = pd.read_csv("Data/TourneyDetailedResults.csv")

### Do some merging/cleaning

#### Add seeds to compact tourney results and change from wteam/lteam to ssteam/wsteam

In [160]:
tourney_results_compact = pd.merge(tourney_results_compact,seeds[['Season','Seed','Team']],left_on=['Season','Wteam'],right_on=['Season','Team'],how='left')
tourney_results_compact=tourney_results_compact.rename(columns = {'Seed':'Wteam_Seed'}).drop('Team',axis=1)
tourney_results_compact = pd.merge(tourney_results_compact,seeds[['Season','Seed','Team']],left_on=['Season','Lteam'],right_on=['Season','Team'],how='left')
tourney_results_compact=tourney_results_compact.rename(columns = {'Seed':'Lteam_Seed'}).drop('Team',axis=1)
#create variable that says whether the higher (strong) seed won or lost. The nested where is because in later rounds, you
# may get the same seed number when teams from different regions play. In those cases, the region coming first alphabeticall
# is considered the strong seed. For example, if X01 and Y01 play, X01 is the strong seed since x < y.
tourney_results_compact.loc[:,'StrongSeed'] = np.where(tourney_results_compact.Wteam_Seed.str[1:3] == tourney_results_compact.Lteam_Seed.str[1:3], 
                                                       np.where(tourney_results_compact.Wteam_Seed <= tourney_results_compact.Lteam_Seed,"Wteam","Lteam"),
                                                       np.where(tourney_results_compact.Wteam_Seed.str[1:] <= tourney_results_compact.Lteam_Seed.str[1:],"Wteam","Lteam"))

#send to separate dataframe depending on whether strong seed won
tourney_strong_wteam = tourney_results_compact[tourney_results_compact.StrongSeed == "Wteam"].copy()
tourney_strong_lteam = tourney_results_compact[tourney_results_compact.StrongSeed == "Lteam"].copy()
#rename columns to SS (strong seed) and WS (weak seed) depending on which won
tourney_strong_wteam = tourney_strong_wteam.rename(columns = {'Wteam':'SS_team','Wscore':'SS_score','Lteam':'WS_team',
                                                              'Lscore':'WS_score','Wloc':'loc','Wteam_Seed':'SS_Seed',
                                                              'Lteam_Seed':'WS_Seed'})
tourney_strong_lteam = tourney_strong_lteam.rename(columns = {'Wteam':'WS_team','Wscore':'WS_score','Lteam':'SS_team',
                                                              'Lscore':'SS_score','Wloc':'loc','Wteam_Seed':'WS_Seed',
                                                              'Lteam_Seed':'SS_Seed'})
#put the two dataframes back together. Now instead of wteam and lteam (which we dont know before the game is played), we have SS and WS (which we know
#when the bracket is announced)
tourney_results_compact = pd.concat([tourney_strong_wteam, tourney_strong_lteam], ignore_index=False).sort_index()
#change seed for winner of play-in games (drop a or b from seed)
tourney_results_compact.loc[:,'SS_Seed'] = np.where(tourney_results_compact.Daynum >=136,tourney_results_compact.SS_Seed.str[0:3],tourney_results_compact.SS_Seed)
tourney_results_compact.loc[:,'WS_Seed'] = np.where(tourney_results_compact.Daynum >=136,tourney_results_compact.WS_Seed.str[0:3],tourney_results_compact.WS_Seed)

### Fill in rest of slots

In [162]:
def get_round_results(previous_round,roundnum):

    current_round = slots[(slots.Round ==roundnum) & (slots.Season < 2017)].copy()
    current_round_long = pd.merge(pd.melt(current_round,id_vars=['Season','Slot'],value_vars=['Strongseed','Weakseed'],value_name='RSlot'),
            previous_round[['Season','Slot','Winning_Seed']],left_on=['Season','RSlot'],right_on=['Season','Slot'],how='left') 
    current_round_long = current_round_long[['Season','Slot_x','Winning_Seed']].copy()
    current_round_long = current_round_long.rename(columns={"Slot_x":"Slot"})
    current_round_long.loc[:,"SeedNum"] = current_round_long.Winning_Seed.str[1:3].astype(int)
    current_round_long.loc[:,"Region"] = current_round_long.Winning_Seed.str[0:1]
    current_round_long = current_round_long.sort_values(['Season','Slot','SeedNum','Region']).drop(['SeedNum','Region'],axis=1)
    current_round_long.loc[:,'Is_WS'] = current_round_long.groupby(['Season','Slot']).cumcount()
    current_round_long_0 = current_round_long[current_round_long.Is_WS == 0].copy().rename(columns={"Winning_Seed":"SS_Seed"}).drop(['Is_WS'],axis=1)
    current_round_long_1 = current_round_long[current_round_long.Is_WS == 1].copy().rename(columns={"Winning_Seed":"WS_Seed"}).drop(['Is_WS'],axis=1)
    current_round = pd.merge(current_round_long_0,current_round_long_1,on=['Season','Slot'])
    current_round.loc[:,"Round"] = roundnum
    return(pd.merge(current_round,tourn_results,on=['Season','SS_Seed','WS_Seed'],how='left'))

In [163]:
tourn_results = tourney_results_compact[tourney_results_compact.Season < 2017].copy()
tourn_results.loc[:,"SS_Win"] = np.where(tourn_results.SS_score >= tourn_results.WS_score,1, 0)
tourn_results.loc[:,"Winning_Seed"] = np.where(tourn_results.SS_Win ==1,
                                               tourn_results.SS_Seed,
                                               tourn_results.WS_Seed)
tourn_results = tourn_results[['Season','SS_Win','Winning_Seed','SS_Seed','WS_Seed','SS_score','WS_score']].copy()

slots_r1 = pd.merge(tourn_results,slots,left_on=['Season','SS_Seed','WS_Seed'],right_on=['Season','Strongseed','Weakseed'],
                              how='inner')
slots_r2 = get_round_results(slots_r1,2)
slots_r3 = get_round_results(slots_r2,3)
slots_r4 = get_round_results(slots_r3,4)
slots_r5 = get_round_results(slots_r4,5)
slots_r6 = get_round_results(slots_r5,6)

slots_complete = pd.concat([slots_r1, slots_r2, slots_r3, slots_r4, slots_r5, slots_r6], 
                           ignore_index=False).reset_index().drop(['index'],axis=1).drop(['Strongseed','Weakseed'],axis=1)


In [164]:
slots_complete = pd.merge(slots_complete,tourney_results_compact[['Season','SS_Seed','WS_Seed','SS_score','WS_score','Daynum','SS_team','WS_team']],
         on=['Season','SS_Seed','WS_Seed','SS_score','WS_score'],how='outer')

#### Add day number for each season

In [165]:
game_data_clean['Date'] = pd.to_datetime(game_data_clean.Date,format='%Y-%m-%d')
#first_dates = game_data_clean.loc[:,['Year','Date']].groupby('Year').min().reset_index()
#first_dates = first_dates.rename(columns = {'Date':'FirstDate'})
first_dates = pd.DataFrame()
first_dates[['FirstDate','Year']] = seasons[['Dayzero','Season']]
game_data_clean = pd.merge(game_data_clean,first_dates,on="Year",how='left')
game_data_clean['Day_Number'] = (game_data_clean['Date'] - game_data_clean['FirstDate']).astype('timedelta64[D]').astype(int)
game_data_clean.drop('FirstDate',axis=1,inplace=True)
game_data_clean.loc[:,"TournamentGame"] = np.where(game_data_clean.Day_Number < 134,0,1)

#### Create dataframe that is all possible matchups in 2017.

In [166]:
slots_2017 = slots[slots.Season == 2017].copy()
seeds_2017 = seeds[seeds.Season == 2017].copy()

In [167]:
teams_in_2017 = seeds_2017[["Team",'Seed','SeedNum','Region']].copy()
teams_in_2017 = teams_in_2017.reset_index().drop('index',axis=1)
teams_in_2017['New_ID'] = 1

all_matchups = pd.merge(teams_in_2017.rename(columns={"Team":"Team1","Seed":"Seed1","SeedNum":"SeedNum1","Region":"Region1"}),
                        teams_in_2017.rename(columns={"Team":"Team2","Seed":"Seed2","SeedNum":"SeedNum2","Region":"Region2"}),
                        on='New_ID',
                        how='outer').drop('New_ID',axis=1)
all_matchups = all_matchups[all_matchups.Seed1 != all_matchups.Seed2]
all_matchups.loc[:,'SS_Seed'] = np.where(all_matchups.Seed1.str[1:3] == all_matchups.Seed2.str[1:3], 
                                                       np.where(all_matchups.Seed1 <= all_matchups.Seed2,"Seed1","Seed2"),
                                                       np.where(all_matchups.Seed1.str[1:] <= all_matchups.Seed2.str[1:],"Seed1","Seed2"))

#send to separate dataframe depending on whether strong seed won
tourney_strong_Seed1 = all_matchups[all_matchups.SS_Seed == "Seed1"].copy().drop(['SeedNum1','Region1','SeedNum2','Region2','SS_Seed'],axis=1)
tourney_strong_Seed2 = all_matchups[all_matchups.SS_Seed == "Seed2"].copy().drop(['SeedNum1','Region1','SeedNum2','Region2','SS_Seed'],axis=1)
#rename columns to SS (strong seed) and WS (weak seed) depending on which won
tourney_strong_Seed1 = tourney_strong_Seed1.rename(columns = {'Team1':'SS_team','Team2':'WS_team','Seed1':'SS_Seed','Seed2':'WS_Seed'})
tourney_strong_Seed2 = tourney_strong_Seed2.rename(columns = {'Team1':'WS_team','Team2':'SS_team','Seed1':'WS_Seed','Seed2':'SS_Seed'})
#put the two dataframes back together. Now instead of Seed1 and Seed2 (which we dont know before the game is played), we have SS and WS (which we know
#when the bracket is announced)
all_matchups_2017 = pd.concat([tourney_strong_Seed1, tourney_strong_Seed2], ignore_index=False).sort_index()


### Create separate datasets for 2017

In [168]:
y2017_game_data = game_data_clean[game_data_clean.Year == 2017].copy()
game_data_clean = game_data_clean[game_data_clean.Year < 2017].copy()
y2017_team_data = team_data_clean[team_data_clean.Year == 2017].copy()
team_data_clean = team_data_clean[team_data_clean.Year < 2017].copy()

# Save dataframes to SQL database

In [169]:
conn = sqlite3.connect("Data/clean_basketball_data.db")
slots_2017.to_sql("slots_2017",conn,if_exists="replace")
all_matchups_2017.to_sql("matchups_2017",conn,if_exists="replace")
y2017_game_data.to_sql("game_data_2017",conn,if_exists="replace")
y2017_team_data.to_sql("team_data_2017",conn,if_exists="replace")
game_data_clean.to_sql("game_data_clean",conn,if_exists="replace")
team_data_clean.to_sql("team_data_clean",conn,if_exists="replace")
name_xwalk.to_sql("team_names",conn,if_exists="replace")
seasons.to_sql("seasons",conn,if_exists="replace")
seeds.to_sql("seeds",conn,if_exists="replace")
season_results_compact.to_sql("season_results_compact",conn,if_exists="replace")
season_results_detailed.to_sql("season_results_detailed",conn,if_exists="replace")
tourney_results_compact.to_sql("tourney_results_compact",conn,if_exists="replace")
tourney_results_detailed.to_sql("tourney_results_detailed",conn,if_exists="replace")
slots_complete.to_sql("tourney_results_complete",conn,if_exists="replace")
slots.to_sql("slots",conn,if_exists="replace")
conn.close()  